In [2]:
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import scipy
from sentence_transformers import SentenceTransformer, util
import json
import pandas as pd
import re
from tqdm import tqdm
import jieba

/home/wuyuman/anaconda3/envs/chatglm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

f = open("/home/wuyuman/Chatglm/elements.json",encoding='utf-8')
content = f.read()
elements = json.loads(content)
f = open("/home/wuyuman/Chatglm/article_all.json",encoding='utf-8')
content = f.read()
articles = json.loads(content)
data=pd.read_csv('/home/wuyuman/Chatglm/test_data_all.csv')
accu=open('/home/wuyuman/Chatglm/data/new_accu.txt',encoding='gbk').readlines()
accu = [i.replace('\n', '') for i in accu]

all_data=pd.read_csv('/home/wuyuman/Chatglm/fact_selected.csv')
f = open("/home/wuyuman/Chatglm/article_selected.json",encoding='utf-8')
content = f.read()
article_selected = json.loads(content)

In [3]:
article_content=[]
crime_definition=[]
definition_and_article=[]
crimes=[]

for accusation in article_selected:
    article_content.append(accusation+':'+article_selected[accusation]['内容'])
    crime_definition.append(elements[accusation[:-1]][0])
    definition_and_article.append(elements[accusation[:-1]][0]+article_selected[accusation]['内容'])
    crimes.append(accusation)

In [6]:
stopwords = open('/home/wuyuman/Chatglm/hit_stopwords.txt').read().split('\n')
def clean_token(sentence_tokens):
    tokens = []
    for word in sentence_tokens:
      if word in stopwords:
        continue
      else:
        tokens.append(word)
    return tokens

In [38]:
stopwords = open('/home/wuyuman/Chatglm/hit_stopwords.txt').read().split('\n')

In [69]:
def bm25_retrivel(fact_data,knowledge,if_clean):
    data=fact_data.copy()
    doc_list = [doc for doc in knowledge if doc != '']
    tokenized_corpus = []
    retrivel_top10=[]
    retrivel_top5=[]
    retrivel_top3=[]
    stopwords = open('/home/wuyuman/Chatglm/hit_stopwords.txt').read().split('\n') 
           
    for sentence in doc_list:
        sentence_words = jieba.lcut(sentence)
        if if_clean==1:
            tokenized_corpus.append(clean_token(sentence_words))
        else:
            tokenized_corpus.append(sentence_words)

    bm25 = BM25Okapi(tokenized_corpus)
    for query in tqdm(data['fact']):
        tokenized_query = jieba.lcut(query)
        if if_clean==1:
            tokenized_query=clean_token(tokenized_query)
        retrivel_top10.append([''.join(i) for i in bm25.get_top_n(tokenized_query, tokenized_corpus, n=10)])
        retrivel_top5.append([''.join(i) for i in bm25.get_top_n(tokenized_query, tokenized_corpus, n=5)])
        retrivel_top3.append([''.join(i) for i in bm25.get_top_n(tokenized_query, tokenized_corpus, n=3)])
    data['bm25_top5']=[[j.split('罪')[0]+'罪' for j in i]  for i in retrivel_top5]
    data['bm25_top10']=[[j.split('罪')[0]+'罪' for j in i]  for i in retrivel_top10]
    data['bm25_top3']=[[j.split('罪')[0]+'罪' for j in i]  for i in retrivel_top3]

    return data

In [70]:
data1=bm25_retrivel(fact_data=all_data,knowledge=article_content,if_clean=1)
data2=bm25_retrivel(fact_data=all_data,knowledge=crime_definition,if_clean=1)
data3=bm25_retrivel(fact_data=all_data,knowledge=definition_and_article,if_clean=1)

100%|██████████| 13994/13994 [04:13<00:00, 55.21it/s]


In [82]:
data10=bm25_retrivel(fact_data=all_data,knowledge=article_content,if_clean=0)
data20=bm25_retrivel(fact_data=all_data,knowledge=crime_definition,if_clean=0)
data30=bm25_retrivel(fact_data=all_data,knowledge=definition_and_article,if_clean=0)

100%|██████████| 13994/13994 [04:31<00:00, 51.57it/s]


In [86]:
data1.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_article.csv')
data2.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_crime.csv')
data3.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_article_and_crime.csv')

In [87]:
data1.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_article_stop.csv')
data2.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_crime_stop.csv')
data3.to_csv('/home/wuyuman/Chatglm/data/retrivel_data/bm25_article_and_crime_stop.csv')

In [71]:
data1[['accusation','bm25_top3','bm25_top5','bm25_top10']]

,accusation,bm25_top3,bm25_top5,bm25_top10
0,盗窃罪,"[非国家工作人员受贿罪, 违法发放贷款罪, 走私贩卖运输制造毒品罪]","[非国家工作人员受贿罪, 违法发放贷款罪, 走私贩卖运输制造毒品罪, 非国家工作人员行贿罪,...","[非国家工作人员受贿罪, 违法发放贷款罪, 走私贩卖运输制造毒品罪, 非国家工作人员行贿罪,..."
1,盗窃罪,"[伪证罪, 妨害公务罪, 挪用资金罪]","[伪证罪, 妨害公务罪, 挪用资金罪, 挪用公款罪, 非法经营罪]","[伪证罪, 妨害公务罪, 挪用资金罪, 挪用公款罪, 非法经营罪, 危险驾驶罪, 强奸罪, ..."
2,盗窃罪,"[强奸罪, 非法处置查封扣押冻结财产罪, 伪证罪]","[强奸罪, 非法处置查封扣押冻结财产罪, 伪证罪, 挪用资金罪, 盗掘古文化遗址古墓葬罪]","[强奸罪, 非法处置查封扣押冻结财产罪, 伪证罪, 挪用资金罪, 盗掘古文化遗址古墓葬罪, ..."
3,盗窃罪,"[强奸罪, 伪证罪, 盗窃罪]","[强奸罪, 伪证罪, 盗窃罪, 挪用资金罪, 盗掘古文化遗址古墓葬罪]","[强奸罪, 伪证罪, 盗窃罪, 挪用资金罪, 盗掘古文化遗址古墓葬罪, 妨害公务罪, 挪用公..."
4,盗窃罪,"[非法采矿罪, 制作复制出版贩卖传播淫秽物品牟利罪, 滥伐林木罪]","[非法采矿罪, 制作复制出版贩卖传播淫秽物品牟利罪, 滥伐林木罪, 盗伐林木罪, 非法收购运...","[非法采矿罪, 制作复制出版贩卖传播淫秽物品牟利罪, 滥伐林木罪, 盗伐林木罪, 非法收购运..."
...,...,...,...,...
13989,强迫交易罪,"[挪用资金罪, 贪污罪, 职务侵占罪]","[挪用资金罪, 贪污罪, 职务侵占罪, 非国家工作人员受贿罪, 强迫交易罪]","[挪用资金罪, 贪污罪, 职务侵占罪, 非国家工作人员受贿罪, 强迫交易罪, 伪造公司企业事..."
13990,妨害公务罪,"[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪]","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪, 伪证罪, 非...","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪, 伪证罪, 非..."
13991,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 挪用资金罪]","[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 挪用资金罪, 非法采矿罪, 掩饰隐...","[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 挪用资金罪, 非法采矿罪, 掩饰隐..."
13992,窝藏、包庇罪,"[伪证罪, 非法行医罪, 非法进行节育手术罪]","[伪证罪, 非法行医罪, 非法进行节育手术罪, 挪用资金罪, 绑架罪]","[伪证罪, 非法行医罪, 非法进行节育手术罪, 挪用资金罪, 绑架罪, 强奸罪, 交通肇事罪..."


In [72]:
data2[['accusation','bm25_top3','bm25_top5','bm25_top10']]

,accusation,bm25_top3,bm25_top5,bm25_top10
0,盗窃罪,"[盗窃罪, 盗掘古文化遗址古墓葬罪, 非法采矿罪]","[盗窃罪, 盗掘古文化遗址古墓葬罪, 非法采矿罪, 故意杀人罪, 行贿罪]","[盗窃罪, 盗掘古文化遗址古墓葬罪, 非法采矿罪, 故意杀人罪, 行贿罪, 出售购买运输假币..."
1,盗窃罪,"[伪证罪, 敲诈勒索罪, 挪用资金罪]","[伪证罪, 敲诈勒索罪, 挪用资金罪, 挪用公款罪, 妨害公务罪]","[伪证罪, 敲诈勒索罪, 挪用资金罪, 挪用公款罪, 妨害公务罪, 非法经营罪, 盗掘古文化..."
2,盗窃罪,"[敲诈勒索罪, 盗掘古文化遗址古墓葬罪, 挪用资金罪]","[敲诈勒索罪, 盗掘古文化遗址古墓葬罪, 挪用资金罪, 非法处置查封扣押冻结财产罪, 非法种...","[敲诈勒索罪, 盗掘古文化遗址古墓葬罪, 挪用资金罪, 非法处置查封扣押冻结财产罪, 非法种..."
3,盗窃罪,"[挪用公款罪, 敲诈勒索罪, 盗掘古文化遗址古墓葬罪]","[挪用公款罪, 敲诈勒索罪, 盗掘古文化遗址古墓葬罪, 挪用资金罪, 非法侵入住宅罪]","[挪用公款罪, 敲诈勒索罪, 盗掘古文化遗址古墓葬罪, 挪用资金罪, 非法侵入住宅罪, 伪证..."
4,盗窃罪,"[非法侵入住宅罪, 非法采矿罪, 敲诈勒索罪]","[非法侵入住宅罪, 非法采矿罪, 敲诈勒索罪, 挪用资金罪, 拐骗儿童罪]","[非法侵入住宅罪, 非法采矿罪, 敲诈勒索罪, 挪用资金罪, 拐骗儿童罪, 盗掘古文化遗址古..."
...,...,...,...,...
13989,强迫交易罪,"[伪造公司企业事业单位人民团体印章罪, 贪污罪, 挪用资金罪]","[伪造公司企业事业单位人民团体印章罪, 贪污罪, 挪用资金罪, 违法发放贷款罪, 挪用公款罪]","[伪造公司企业事业单位人民团体印章罪, 贪污罪, 挪用资金罪, 违法发放贷款罪, 挪用公款罪..."
13990,妨害公务罪,"[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 伪证罪]","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 伪证罪, 非法进行节育手术罪, 挪用公款罪]","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 伪证罪, 非法进行节育手术罪, 挪用公款罪..."
13991,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"[盗掘古文化遗址古墓葬罪, 非法采矿罪, 掩饰隐瞒犯罪]","[盗掘古文化遗址古墓葬罪, 非法采矿罪, 掩饰隐瞒犯罪, 非法处置查封扣押冻结财产罪, 挪用...","[盗掘古文化遗址古墓葬罪, 非法采矿罪, 掩饰隐瞒犯罪, 非法处置查封扣押冻结财产罪, 挪用..."
13992,窝藏、包庇罪,"[伪证罪, 非法行医罪, 挪用公款罪]","[伪证罪, 非法行医罪, 挪用公款罪, 敲诈勒索罪, 窝藏包庇罪]","[伪证罪, 非法行医罪, 挪用公款罪, 敲诈勒索罪, 窝藏包庇罪, 遗弃罪, 挪用资金罪, ..."


In [73]:
data3[['accusation','bm25_top3','bm25_top5','bm25_top10']]

,accusation,bm25_top3,bm25_top5,bm25_top10
0,盗窃罪,"[盗窃罪, 非国家工作人员受贿罪, 违法发放贷款罪]","[盗窃罪, 非国家工作人员受贿罪, 违法发放贷款罪, 盗掘古文化遗址古墓葬罪, 走私贩卖运输...","[盗窃罪, 非国家工作人员受贿罪, 违法发放贷款罪, 盗掘古文化遗址古墓葬罪, 走私贩卖运输..."
1,盗窃罪,"[伪证罪, 妨害公务罪, 挪用公款罪]","[伪证罪, 妨害公务罪, 挪用公款罪, 挪用资金罪, 非法经营罪]","[伪证罪, 妨害公务罪, 挪用公款罪, 挪用资金罪, 非法经营罪, 敲诈勒索罪, 非法处置查..."
2,盗窃罪,"[敲诈勒索罪, 非法处置查封扣押冻结财产罪, 强奸罪]","[敲诈勒索罪, 非法处置查封扣押冻结财产罪, 强奸罪, 伪证罪, 挪用资金罪]","[敲诈勒索罪, 非法处置查封扣押冻结财产罪, 强奸罪, 伪证罪, 挪用资金罪, 盗掘古文化遗..."
3,盗窃罪,"[挪用公款罪, 敲诈勒索罪, 强奸罪]","[挪用公款罪, 敲诈勒索罪, 强奸罪, 伪证罪, 挪用资金罪]","[挪用公款罪, 敲诈勒索罪, 强奸罪, 伪证罪, 挪用资金罪, 盗掘古文化遗址古墓葬罪, 妨..."
4,盗窃罪,"[非法侵入住宅罪, 非法采矿罪, 挪用资金罪]","[非法侵入住宅罪, 非法采矿罪, 挪用资金罪, 制作复制出版贩卖传播淫秽物品牟利罪, 滥伐林木罪]","[非法侵入住宅罪, 非法采矿罪, 挪用资金罪, 制作复制出版贩卖传播淫秽物品牟利罪, 滥伐林..."
...,...,...,...,...
13989,强迫交易罪,"[挪用资金罪, 贪污罪, 职务侵占罪]","[挪用资金罪, 贪污罪, 职务侵占罪, 非国家工作人员受贿罪, 伪造公司企业事业单位人民团体...","[挪用资金罪, 贪污罪, 职务侵占罪, 非国家工作人员受贿罪, 伪造公司企业事业单位人民团体..."
13990,妨害公务罪,"[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪]","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪, 伪证罪, 非...","[妨害公务罪, 聚众扰乱公共场所秩序交通秩序罪, 编造故意传播虚假恐怖信息罪, 伪证罪, 非..."
13991,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 非法采矿罪]","[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 非法采矿罪, 掩饰隐瞒犯罪, 挪用...","[盗掘古文化遗址古墓葬罪, 非法处置查封扣押冻结财产罪, 非法采矿罪, 掩饰隐瞒犯罪, 挪用..."
13992,窝藏、包庇罪,"[伪证罪, 挪用资金罪, 非法行医罪]","[伪证罪, 挪用资金罪, 非法行医罪, 挪用公款罪, 非法进行节育手术罪]","[伪证罪, 挪用资金罪, 非法行医罪, 挪用公款罪, 非法进行节育手术罪, 强奸罪, 绑架罪..."


In [5]:
def retrievel_acc(data,column1,column2,column3):
    acc1=0
    acc2=0
    acc3=0
    for i in range(len(data)):
        if data['accusation'].iloc[i] in data[column1].iloc[i]:
            acc1+=1
        if data['accusation'].iloc[i] in data[column2].iloc[i]:
            acc2+=1
        if data['accusation'].iloc[i] in data[column3].iloc[i]:
            acc3+=1
    return column1+':'+str(acc1),column2+':'+str(acc2),column3+':'+str(acc3)

In [6]:
def each_acc(data,column):
    acc_dict={}
    num_dict={}
    for i in data['accusation'].value_counts().index:
        data_new=data[data['accusation']==i]
        sum=0
        for j in range(len(data_new)):
            if data_new['accusation'].iloc[j] in data_new[column].iloc[j]:
                sum+=1
        acc_dict[i]=sum/len(data_new)
    return acc_dict

In [76]:
retrievel_acc(data1,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:4100', 'bm25_top5:4813', 'bm25_top10:5715')

In [77]:
retrievel_acc(data2,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:4407', 'bm25_top5:5220', 'bm25_top10:6347')

In [78]:
retrievel_acc(data3,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:4622', 'bm25_top5:5486', 'bm25_top10:6510')

In [83]:
retrievel_acc(data10,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:5709', 'bm25_top5:6584', 'bm25_top10:7753')

In [84]:
retrievel_acc(data20,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:5167', 'bm25_top5:5962', 'bm25_top10:6989')

In [85]:
retrievel_acc(data30,column1='bm25_top3',column2='bm25_top5',column3='bm25_top10')

('bm25_top3:6113', 'bm25_top5:7099', 'bm25_top10:8522')

In [91]:
each_acc(data=data,column='bm25_top5')

{'盗窃罪': 0.05408805031446541,
 '走私、贩卖、运输、制造毒品罪': 0.8904639175257731,
 '故意伤害罪': 0.01282051282051282,
 '诈骗罪': 0.16119402985074627,
 '寻衅滋事罪': 0.18831168831168832,
 '抢劫罪': 0.006825938566552901,
 '危险驾驶罪': 0.967741935483871,
 '合同诈骗罪': 0.42913385826771655,
 '故意杀人罪': 0.0136986301369863,
 '非法占用农用地罪': 0.8529411764705882,
 '信用卡诈骗罪': 0.7931034482758621,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.9801980198019802,
 '强奸罪': 0.9257425742574258,
 '滥伐林木罪': 0.9195979899497487,
 '非法持有、私藏枪支、弹药罪': 0.8442211055276382,
 '交通肇事罪': 0.43147208121827413,
 '非法经营罪': 0.1958762886597938,
 '生产、销售有毒、有害食品罪': 0.8549222797927462,
 '过失致人死亡罪': 0.08808290155440414,
 '生产、销售不符合安全标准的食品罪': 0.609375,
 '污染环境罪': 0.8125,
 '非法行医罪': 0.71875,
 '非法种植毒品原植物罪': 1.0,
 '假冒注册商标罪': 0.774869109947644,
 '非法吸收公众存款罪': 0.4031413612565445,
 '销售假冒注册商标的商品罪': 0.6263157894736842,
 '拒不支付劳动报酬罪': 0.9315789473684211,
 '盗伐林木罪': 0.6157894736842106,
 '拒不执行判决、裁定罪': 0.9523809523809523,
 '故意毁坏财物罪': 0.026455026455026454,
 '职务侵占罪': 0.671957671957672,
 '聚众斗殴罪': 0.09574468085106

In [80]:
each_acc(data=data2,column='bm25_top5')

{'盗窃罪': 0.16855345911949685,
 '走私、贩卖、运输、制造毒品罪': 0.0,
 '故意伤害罪': 0.049679487179487176,
 '诈骗罪': 0.42686567164179107,
 '寻衅滋事罪': 0.2012987012987013,
 '抢劫罪': 0.09897610921501707,
 '危险驾驶罪': 0.992831541218638,
 '合同诈骗罪': 0.21653543307086615,
 '故意杀人罪': 0.3424657534246575,
 '非法占用农用地罪': 0.9019607843137255,
 '信用卡诈骗罪': 0.5714285714285714,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.0,
 '强奸罪': 0.7772277227722773,
 '滥伐林木罪': 0.9547738693467337,
 '非法持有、私藏枪支、弹药罪': 0.0,
 '交通肇事罪': 0.9086294416243654,
 '非法经营罪': 0.3865979381443299,
 '生产、销售有毒、有害食品罪': 0.0,
 '过失致人死亡罪': 0.6373056994818653,
 '生产、销售不符合安全标准的食品罪': 0.0,
 '污染环境罪': 0.6666666666666666,
 '非法行医罪': 0.8177083333333334,
 '非法种植毒品原植物罪': 1.0,
 '假冒注册商标罪': 0.8795811518324608,
 '非法吸收公众存款罪': 0.7015706806282722,
 '销售假冒注册商标的商品罪': 0.0,
 '拒不支付劳动报酬罪': 0.968421052631579,
 '盗伐林木罪': 0.7157894736842105,
 '拒不执行判决、裁定罪': 0.0,
 '故意毁坏财物罪': 0.32275132275132273,
 '职务侵占罪': 0.0,
 '聚众斗殴罪': 0.3670212765957447,
 '行贿罪': 0.6363636363636364,
 '敲诈勒索罪': 0.7967914438502673,
 '窝藏、包庇罪': 0.0,
 '放火罪': 0.17

In [81]:
each_acc(data=data3,column='bm25_top5')

{'盗窃罪': 0.12955974842767295,
 '走私、贩卖、运输、制造毒品罪': 0.0,
 '故意伤害罪': 0.08974358974358974,
 '诈骗罪': 0.16417910447761194,
 '寻衅滋事罪': 0.29545454545454547,
 '抢劫罪': 0.023890784982935155,
 '危险驾驶罪': 0.996415770609319,
 '合同诈骗罪': 0.7598425196850394,
 '故意杀人罪': 0.2922374429223744,
 '非法占用农用地罪': 0.9117647058823529,
 '信用卡诈骗罪': 0.8620689655172413,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.0,
 '强奸罪': 0.900990099009901,
 '滥伐林木罪': 0.9748743718592965,
 '非法持有、私藏枪支、弹药罪': 0.0,
 '交通肇事罪': 0.8324873096446701,
 '非法经营罪': 0.4072164948453608,
 '生产、销售有毒、有害食品罪': 0.0,
 '过失致人死亡罪': 0.17098445595854922,
 '生产、销售不符合安全标准的食品罪': 0.0,
 '污染环境罪': 0.6979166666666666,
 '非法行医罪': 0.8697916666666666,
 '非法种植毒品原植物罪': 1.0,
 '假冒注册商标罪': 0.8586387434554974,
 '非法吸收公众存款罪': 0.6858638743455497,
 '销售假冒注册商标的商品罪': 0.0,
 '拒不支付劳动报酬罪': 0.9736842105263158,
 '盗伐林木罪': 0.8052631578947368,
 '拒不执行判决、裁定罪': 0.0,
 '故意毁坏财物罪': 0.24338624338624337,
 '职务侵占罪': 0.6243386243386243,
 '聚众斗殴罪': 0.2978723404255319,
 '行贿罪': 0.5882352941176471,
 '敲诈勒索罪': 0.6524064171122995,
 '窝藏、包庇罪': 0

In [47]:
acc_dict_sorted=sorted(acc_dict.items(), key=lambda x: x[1], reverse=True)

In [49]:
acc_dict_sorted[-10:]

[('过失致人死亡罪', 0.010362694300518135),
 ('故意伤害罪', 0.003205128205128205),
 ('盗窃罪', 0.0012578616352201257),
 ('诈骗罪', 0.0),
 ('抢劫罪', 0.0),
 ('故意杀人罪', 0.0),
 ('敲诈勒索罪', 0.0),
 ('掩饰、隐瞒犯罪所得、犯罪所得收益罪', 0.0),
 ('抢夺罪', 0.0),
 ('帮助犯罪分子逃避处罚罪', 0.0)]

In [3]:
data1=pd.read_csv('/home/wuyuman/Chatglm/data/retrivel_data/sbert_retrivel1.csv')

In [4]:
data1[['fact','accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,fact,accusation,top_5_article,top_5_crime,top_5_definition_and_article
0,公诉机关起诉指控，被告人张某某秘密窃取他人财物，价值2210元，××数额较大，其行为已触犯《...,盗窃罪,"['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '职务侵占罪', '对...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '侵犯著作权罪', '...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '职务侵占罪', '抢夺罪', '非..."
1,孝昌县人民检察院指控：2014年1月4日，被告人邬某在孝昌县城区2路公交车上××被害人晏某白...,盗窃罪,"['交通肇事罪', '诬告陷害罪', '票据诈骗罪', '寻衅滋事罪', '诈骗罪']","['交通肇事罪', '寻衅滋事罪', '诬告陷害罪', '非法处置查封、扣押、冻结的财产罪'...","['诬告陷害罪', '交通肇事罪', '票据诈骗罪', '单位行贿罪', '寻衅滋事罪']"
2,广东省广州市南沙区人民检察院指控被告人罗某于2015年6月2日到广州市南沙区大岗镇人民路宇航...,盗窃罪,"['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢劫罪', '伪证罪']","['交通肇事罪', '盗窃罪', '抢夺罪', '抢劫罪', '寻衅滋事罪']","['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢夺罪', '冒充军人招摇撞骗罪']"
3,公诉机关指控，2016年3月3日18时许，被告人易某某行至达州市通川区大观园公交车站附近，扒...,盗窃罪,"['伪证罪', '玩忽职守罪', '诬告陷害罪', '招摇撞骗罪', '非法处置查封、扣押、...","['非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '寻衅滋事罪', '妨害公务罪...","['冒充军人招摇撞骗罪', '非法处置查封、扣押、冻结的财产罪', '持有、使用假币罪', ..."
4,公诉机关指控：1.2015年8月20日晚上，被告人胡某甲至杭州市淳安县千岛湖镇新安东路112...,盗窃罪,"['非法吸收公众存款罪', '出售、购买、运输假币罪', '非法处置查封、扣押、冻结的财产罪...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '抢夺罪', '妨害公务...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '出售、购买、运输假币罪..."
...,...,...,...,...,...
13989,深圳市南山区人民检察院指控：深圳市凯2某物业发展有限公司是国有凯2某实业股份有限公司（以下简...,强迫交易罪,"['伪造公司、企业、事业单位、人民团体印章罪', '职务侵占罪', '伪造、变造、买卖国家机...","['伪造公司、企业、事业单位、人民团体印章罪', '伪证罪', '贪污罪', '非法采矿罪'...","['伪造公司、企业、事业单位、人民团体印章罪', '职务侵占罪', '伪证罪', '盗窃罪'..."
13990,经审理查明：2016年11月16日18时许，被告人于某某等人到盖州市人民政府上访，盖州市公安...,妨害公务罪,"['非法拘禁罪', '妨害公务罪', '冒充军人招摇撞骗罪', '玩忽职守罪', '帮助犯罪...","['妨害公务罪', '非法拘禁罪', '聚众扰乱公共场所秩序、交通秩序罪', '帮助犯罪分子...","['妨害公务罪', '非法拘禁罪', '冒充军人招摇撞骗罪', '单位行贿罪', '帮助犯罪..."
13991,新疆生产建设兵团五家渠垦区人民检察院指控，2016年11月23日凌晨3时许，李某（另案起诉）...,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"['走私、贩卖、运输、制造毒品罪', '生产、销售伪劣产品罪', '出售、购买、运输假币罪'...","['非法收购、运输盗伐、滥伐的林木罪', '生产、销售伪劣产品罪', '走私、贩卖、运输、制...","['走私、贩卖、运输、制造毒品罪', '非法收购、运输盗伐、滥伐的林木罪', '冒充军人招摇..."
13992,经审理查明，1994年12月1日13时许，被告人张某与同村村民李某（已判刑）、陈xx及被害人...,窝藏、包庇罪,"['寻衅滋事罪', '故意伤害罪', '聚众斗殴罪', '招摇撞骗罪', '非法拘禁罪']","['寻衅滋事罪', '行贿罪', '聚众斗殴罪', '非法处置查封、扣押、冻结的财产罪', ...","['寻衅滋事罪', '招摇撞骗罪', '故意伤害罪', '聚众斗殴罪', '冒充军人招摇撞骗罪']"


In [59]:
acc_dict_sorted=sorted(acc_dict.items(), key=lambda x: x[1], reverse=True)

In [61]:
acc_dict_sorted[-20:]

[('敲诈勒索罪', 0.19786096256684493),
 ('非法拘禁罪', 0.1871345029239766),
 ('合同诈骗罪', 0.18503937007874016),
 ('过失致人重伤罪', 0.18181818181818182),
 ('重大责任事故罪', 0.1724137931034483),
 ('遗弃罪', 0.16666666666666666),
 ('故意毁坏财物罪', 0.15873015873015872),
 ('过失致人死亡罪', 0.15025906735751296),
 ('赌博罪', 0.1391304347826087),
 ('抢夺罪', 0.13636363636363635),
 ('窝藏、包庇罪', 0.13440860215053763),
 ('破坏易燃易爆设备罪', 0.1111111111111111),
 ('强迫交易罪', 0.1111111111111111),
 ('介绍贿赂罪', 0.1111111111111111),
 ('破坏生产经营罪', 0.05),
 ('掩饰、隐瞒犯罪所得、犯罪所得收益罪', 0.04395604395604396),
 ('绑架罪', 0.03773584905660377),
 ('贪污罪', 0.03468208092485549),
 ('投放危险物质罪', 0.029411764705882353),
 ('非法经营罪', 0.015463917525773196)]

In [62]:
data1[data1['accusation']=='盗窃罪'][['fact','accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,fact,accusation,top_5_article,top_5_crime,top_5_definition_and_article
0,公诉机关起诉指控，被告人张某某秘密窃取他人财物，价值2210元，××数额较大，其行为已触犯《...,盗窃罪,"['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '职务侵占罪', '对...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '侵犯著作权罪', '...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '职务侵占罪', '抢夺罪', '非..."
1,孝昌县人民检察院指控：2014年1月4日，被告人邬某在孝昌县城区2路公交车上××被害人晏某白...,盗窃罪,"['交通肇事罪', '诬告陷害罪', '票据诈骗罪', '寻衅滋事罪', '诈骗罪']","['交通肇事罪', '寻衅滋事罪', '诬告陷害罪', '非法处置查封、扣押、冻结的财产罪'...","['诬告陷害罪', '交通肇事罪', '票据诈骗罪', '单位行贿罪', '寻衅滋事罪']"
2,广东省广州市南沙区人民检察院指控被告人罗某于2015年6月2日到广州市南沙区大岗镇人民路宇航...,盗窃罪,"['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢劫罪', '伪证罪']","['交通肇事罪', '盗窃罪', '抢夺罪', '抢劫罪', '寻衅滋事罪']","['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢夺罪', '冒充军人招摇撞骗罪']"
3,公诉机关指控，2016年3月3日18时许，被告人易某某行至达州市通川区大观园公交车站附近，扒...,盗窃罪,"['伪证罪', '玩忽职守罪', '诬告陷害罪', '招摇撞骗罪', '非法处置查封、扣押、...","['非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '寻衅滋事罪', '妨害公务罪...","['冒充军人招摇撞骗罪', '非法处置查封、扣押、冻结的财产罪', '持有、使用假币罪', ..."
4,公诉机关指控：1.2015年8月20日晚上，被告人胡某甲至杭州市淳安县千岛湖镇新安东路112...,盗窃罪,"['非法吸收公众存款罪', '出售、购买、运输假币罪', '非法处置查封、扣押、冻结的财产罪...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '抢夺罪', '妨害公务...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '出售、购买、运输假币罪..."
...,...,...,...,...,...
2178,经审理查明：1、2015年7月9日14时许，被告人孙1某在本市涧西区中州西路附近的中通快递涧...,盗窃罪,"['盗窃罪', '寻衅滋事罪', '抢劫罪', '敲诈勒索罪', '出售、购买、运输假币罪']","['盗窃罪', '抢劫罪', '非法处置查封、扣押、冻结的财产罪', '非法收购、运输盗伐、...","['盗窃罪', '寻衅滋事罪', '敲诈勒索罪', '抢劫罪', '非法处置查封、扣押、冻结..."
2179,资兴市人民检察院指控：2014年7月1日晚，被告人陈某某在资兴市鲤鱼江雄森大酒店旁××被害人...,盗窃罪,"['信用卡诈骗罪', '集资诈骗罪', '票据诈骗罪', '持有、使用假币罪', '诈骗罪']","['骗取贷款、票据承兑、金融票证罪', '持有、使用假币罪', '信用卡诈骗罪', '诈骗罪...","['信用卡诈骗罪', '集资诈骗罪', '票据诈骗罪', '持有、使用假币罪', '贷款诈骗罪']"
2180,河南省舞钢市人民检察院指控，2015年6月1日20时许，被告人李1某在舞钢市朱兰干休路南段兰...,盗窃罪,"['招摇撞骗罪', '滥用职权罪', '伪证罪', '诬告陷害罪', '非法拘禁罪']","['非法处置查封、扣押、冻结的财产罪', '行贿罪', '招摇撞骗罪', '伪证罪', '交...","['招摇撞骗罪', '冒充军人招摇撞骗罪', '单位行贿罪', '诬告陷害罪', '非法拘禁罪']"
2181,公诉机关指控，2014年12月6日17时许，被告人张某在安陆市府城办事处外贸十字路口附近趁被...,盗窃罪,"['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '诈骗罪', '寻衅滋事罪', '伪...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '抢夺罪', '...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '票据诈骗罪',..."


In [63]:
data1[(data1['accusation']=='盗窃罪') & (data1['accusation'].isin(data1['top_5_article']))][['fact','accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,fact,accusation,top_5_article,top_5_crime,top_5_definition_and_article
